## 데이터 전처리

### 실습 데이터셋 안내

- 2021년 1~12월에 전세계 165여 개 국가와 수산물 수출입 내역입니다.<br><br>

- 변수(열)에 대한 간단한 설명입니다.
  - stdYymm: 기준년월(yyyymm)
  - nationCode: 국가코드
  - mprcExipitmCode: 수산물수출입품목코드(10자리 숫자)
  - imxprtSeCode: 수출입구분코드(E, I)
  - nationNm: 국가명
  - ecoblNm: 경제권명
  - mprcExipitmNm: 수산물수출입품목명
  - imxprtSeNm: 수출입구분명(수출, 수입)
  - imxprtWt: 수출입중량(kg)
  - imxprtDollarAmount: 수출입금액($)<br><br>

- 실습 데이터인 csv 파일을 아래 링크로 제공합니다.
  - https://bit.ly/Seafood_Trade_2021_CP949
  - 구분자는 콤마(,)이고 인코딩방식은 CP949입니다.

### 관련 라이브러리 호출

In [ ]:
# 관련 라이브러리를 호출합니다.
import requests
import chardet
import numpy as np
import pandas as pd
import os
import joblib

### 인코딩 방식 확인

In [ ]:
# 온라인으로 공유 중인 csv 파일 주소를 변수에 할당합니다.
url = 'https://bit.ly/Seafood_Trade_2021_CP949'

In [ ]:
# 링크에 있는 텍스트를 내려받습니다.(HTTP Requests)
# [힌트] requests.get() 함수를 사용하세요.
res = requests.get(url = url)

In [ ]:
# res에서 바이너리 텍스트의 일부를 출력합니다.
# [힌트] content 속성을 사용하세요.
res.content[:200]

In [ ]:
# 바이너리 텍스트의 인코딩 방식을 확인합니다.
# [힌트] chardet.detect() 함수를 사용하세요.
chardet.detect(res.content)

### 실습 데이터셋 준비

In [ ]:
# 링크를 읽고 데이터프레임 sfd를 생성합니다.
# [힌트] pd.read_csv() 함수를 사용하세요.
# [참고] EUC-KR은 CP949의 부분집합입니다.
sfd = pd.read_csv(filepath_or_buffer = url, encoding = 'CP949')

In [ ]:
# sfd의 정보를 확인합니다.
# [힌트] info() 함수를 사용하세요.
sfd.info()

In [ ]:
# sfd를 출력합니다.
sfd

In [ ]:
# sfd의 처음 5행을 출력합니다.
# [힌트] head() 함수를 사용하세요.
sfd.head()

In [ ]:
# sfd의 마지막 5행을 출력합니다.
# [힌트] tail() 함수를 사용하세요.
sfd.tail()

### 실습 데이터셋 전처리

In [ ]:
# sfd의 열이름을 출력합니다.
sfd.columns

In [ ]:
# sfd에서 'stdYymm', 'nationNm', 'mprcExipitmNm', 'imxprtSeNm', 'imxprtWt', 'imxprtDollarAmount'를
# 선택하기 위해 열이름을 원소로 갖는 리스트를 변수에 할당합니다.
cols = ['stdYymm', 'nationNm', 'mprcExipitmNm', 'imxprtSeNm', 'imxprtWt', 'imxprtDollarAmount']

In [ ]:
# sfd에서 cols만 선택한 결과를 출력합니다.
sfd[cols]

In [ ]:
# 인덱싱 결과를 sfd에 재할당합니다.
sfd = sfd[cols]

In [ ]:
# sfd의 전체 열이름을 'month', 'nation', 'mpritm', 'imxprt', 'weight', 'dollar'으로 변경합니다.
sfd.columns = ['month', 'nation', 'mpritm', 'imxprt', 'weight', 'dollar']

In [ ]:
# sfd의 열별 자료형을 확인합니다.
sfd.dtypes

In [ ]:
# month의 자료형을 문자열로 변환하고 month에 할당합니다. 
# [힌트] astype() 함수를 사용하세요.
sfd['month'] = sfd['month'].astype(str)

In [ ]:
# dollar를 1000으로 나눈 결과를 grand에 할당합니다.
# [참고] 데이터프레임 오른쪽 끝에 새 변수를 추가합니다.
sfd['grand'] = sfd['dollar'] / 1000

In [ ]:
# sfd에서 dollar를 삭제한 결과를 출력합니다.
# [힌트] drop() 함수를 사용하세요.
sfd.drop(columns = ['dollar'])

In [ ]:
# sfd에서 dollar를 삭제한 결과를 sfd에 재할당합니다.
sfd = sfd.drop(columns = ['dollar'])

In [ ]:
# sfd의 수치(정수 또는 실수)형 변수의 기술통계량을 확인합니다.
# [힌트] describe() 함수를 사용하세요.
sfd.describe()

In [ ]:
# sfd의 범주형 변수의 기술통계량을 확인합니다.
sfd.describe(include = object)

In [ ]:
# nation의 빈도수를 확인합니다.
# [힌트] value_counts() 함수를 사용하세요.
sfd['nation'].value_counts()

In [ ]:
# nation의 상대도수를 확인합니다.
# [힌트] value_counts() 함수에 normalize 매개변수를 추가하세요.
sfd['nation'].value_counts(normalize = True)

### 수산물품목별 수출입금액을 합산하고 무역수지 계산

In [ ]:
# 아래 조건을 만족하는 피벗 테이블을 mpr에 할당합니다.
# [힌트] pd.pivot_table() 함수를 사용하세요.
# 데이터(data)는 sfd, 값(values)은 grand, 행이름(index)은 mpritm, 
# 열이름(columns)은 imxprt, 집계함수는 np.sum을 지정하세요.
mpr = pd.pivot_table(
    data = sfd, 
    values = 'grand', 
    index = 'mpritm', 
    columns = 'imxprt', 
    aggfunc = np.sum
)

In [ ]:
# mpr의 처음 5행을 출력합니다.
mpr.head()

In [ ]:
# mpr의 행 개수를 출력합니다.
# [힌트] shape 속성 결과에 0번 인덱스 원소를 선택하세요.
mpr.shape[0]

In [ ]:
# mpr의 열별 결측값 개수를 확인합니다.
# [힌트] isna() 함수를 사용하세요.
mpr.isna().sum()

In [ ]:
# mpr에 있는 결측값을 0으로 대체하고 mpr에 재할당합니다.
# [힌트] fillna() 함수를 사용하세요.
mpr = mpr.fillna(value = 0)

In [ ]:
# 수출에서 수입을 뺀 무역수지를 생성합니다.
mpr['무역수지'] = mpr['수출'] - mpr['수입']

In [ ]:
# mpr를 무역수지로 내림차순 정렬하고 mpr에 재할당합니다.
# [힌트] sort_values() 함수를 사용하세요.
mpr = mpr.sort_values(by = ['무역수지'], ascending = False)

In [ ]:
# mpr의 처음 10행을 출력합니다.
mpr.head(n = 10)

In [ ]:
# mpr의 마지막 10행을 출력합니다.
mpr.tail(n = 10)

### 특정 수산물품목에 대한 국가별 수출단가 계산

In [ ]:
# 아래 조건을 만족하는 피벗 테이블을 itm에 할당합니다.
# [힌트] pd.pivot_table() 함수를 사용하세요.
# 데이터(data)는 sfd에서 '김' & '수출' 행을 지정하세요.
# 값(values)은 grand와 weight를 리스트로 지정하세요.
# 행이름(index)은 nation, 집계함수는 np.sum을 지정하세요.
itm = pd.pivot_table(
    data = sfd[sfd['mpritm'].eq('김') & sfd['imxprt'].eq('수출')], 
    values = ['grand', 'weight'], 
    index = 'nation', 
    aggfunc = np.sum
)

In [ ]:
# itm의 처음 5행을 출력합니다.
itm.head()

In [ ]:
# itm의 행 개수를 출력합니다.
itm.shape[0]

In [ ]:
# itm의 열별 결측값 개수를 확인합니다.
itm.isna().sum()

In [ ]:
# itm의 열이름을 '금액', '중량'으로 변경합니다.
itm.columns = ['금액', '중량']

In [ ]:
# 금액을 중량으로 나눈 값에 1000을 곱한 수출단가(달러)를 생성합니다.
itm['수출단가'] = itm['금액'] / itm['중량'] * 1000

In [ ]:
# itm을 수출단가로 내림차순 정렬하고 itm에 재할당합니다.
# [힌트] sort_values() 함수를 사용하세요.
itm = itm.sort_values(by = ['수출단가'], ascending = False)

In [ ]:
# itm의 처음 10행을 출력합니다.
itm.head(n = 10)

In [ ]:
# itm의 마지막 10행을 출력합니다.
itm.tail(n = 10)

### 국가별 수출입금액을 합산하고 무역수지 계산

In [ ]:
# 아래 조건을 만족하는 피벗 테이블을 pvt에 할당합니다.
# [힌트] pd.pivot_table() 함수를 사용하세요.
# 데이터(data)는 sfd, 값(values)은 grand, 행이름(index)은 nation, 
# 열이름(columns)은 imxprt, 집계함수는 np.sum을 지정하세요.
pvt = pd.pivot_table(
    data = sfd, 
    values = 'grand', 
    index = 'nation', 
    columns = 'imxprt', 
    aggfunc = np.sum
)

In [ ]:
# pvt의 처음 5행을 출력합니다.
pvt.head()

In [ ]:
# pvt의 행 개수를 출력합니다.
pvt.shape[0]

In [ ]:
# pvt의 열별 결측값 개수를 확인합니다.
pvt.isna().sum()

In [ ]:
# pvt에 있는 결측값을 0으로 대체하고 pvt에 재할당합니다.
# [힌트] fillna() 함수를 사용하세요.
pvt = pvt.fillna(value = 0)

In [ ]:
# 수출에서 수입을 뺀 무역수지를 생성합니다.
pvt['무역수지'] = pvt['수출'] - pvt['수입']

In [ ]:
# pvt를 무역수지로 내림차순 정렬하고 pvt에 재할당합니다.
# [힌트] sort_values() 함수를 사용하세요.
pvt = pvt.sort_values(by = ['무역수지'], ascending = False)

In [ ]:
# pvt의 처음 5행을 출력합니다.
pvt.head()

In [ ]:
# pvt의 행이름을 출력합니다.
# [힌트] index 속성을 사용하세요.
pvt.index

In [ ]:
# pvt의 행이름 네임을 '국가'로 변경합니다.
# [힌트] index.name 속성을 사용하세요.
pvt.index.name = '국가'

In [ ]:
# pvt의 열이름을 출력합니다.
# [힌트] columns 속성을 사용하세요.
pvt.columns

In [ ]:
# pvt의 열이름 네임을 삭제합니다.
# [힌트] columns.name 속성을 사용하세요.
# [참고] None을 할당하면 객체를 삭제합니다.
pvt.columns.name = None

In [ ]:
# pvt의 처음 5행을 출력합니다.
pvt.head()

In [ ]:
# pvt의 행이름을 초기화한 결과를 출력합니다.
# [힌트] reset_index() 함수를 사용하세요.
# [참고] 기존 행이름을 맨 처음 열로 추가합니다.
# [참고] drop = True를 추가하면 기존 행이름을 삭제합니다.
pvt.reset_index()

In [ ]:
# pvt의 행이름을 초기화하고 pvt에 재할당합니다.
pvt = pvt.reset_index()

In [ ]:
# pvt의 처음 10행을 출력합니다.(무역수지 흑자 국가)
pvt.head(n = 10)

In [ ]:
# pvt의 마지막 10행을 출력합니다.(무역수지 적자 국가)
pvt.tail(n = 10)

## 데이터 병합

### 실습 데이터셋 안내

- 2021년 전세계 국가별 1인당 국민총소득입니다.
  - 출처: 통계청 KOSIS > 국제통계 > 주제별 통계 > 국민계정 > 1인당 국민총소득<br><br>

- 변수(열)에 대한 간단한 설명입니다.
  - nation: 국가명
  - gdp_2019: 2019년 1인당 국민총소득
  - gdp_2020: 2020년 1인당 국민총소득
  - gdp_2021: 2021년 1인당 국민총소득<br><br>

- 실습 데이터인 csv 파일을 아래 링크로 제공합니다.
  - https://bit.ly/GDP_Per_Capita_2021_UTF8
  - 구분자는 콤마(,)이고 인코딩방식은 UTF-8입니다.

In [ ]:
# 온라인으로 공유 중인 csv 파일 주소를 변수에 할당합니다.
url = 'https://bit.ly/GDP_Per_Capita_2021_UTF8'

In [ ]:
# 링크에 있는 텍스트를 내려받습니다.(HTTP Requests)
res = requests.get(url = url)

In [ ]:
# res에서 바이너리 텍스트의 일부를 출력합니다.
res.content[:200]

In [ ]:
# 바이너리 텍스트의 인코딩 방식을 확인합니다.
chardet.detect(res.content[:200])

In [ ]:
# 링크를 읽고 데이터프레임 gdp를 생성합니다.
# [참고] 인코딩 방식이 UTF-8이면 encoding 매개변수를 생략할 수 있습니다.
gdp = pd.read_csv(filepath_or_buffer = url, encoding = 'UTF-8')

In [ ]:
# gdp의 정보를 확인합니다.
gdp.info()

In [ ]:
# gdp의 처음 5행을 출력합니다.
gdp.head()

In [ ]:
# gdp의 열이름을 출력합니다.
gdp.columns

In [ ]:
# gdp에서 열이름이 'gdp_2019', 'gdp_2020'인 열을 삭제합니다.
# [힌트] drop() 함수를 사용하세요.
gdp = gdp.drop(columns = ['gdp_2019', 'gdp_2020'])

In [ ]:
# gdp의 마지막 5행을 출력합니다.
gdp.tail()

In [ ]:
# gdp_2021의 원소가 '-'이면 결측값(np.nan)으로 변경합니다.
# [힌트] np.where()와 eq() 함수를 사용하세요.
gdp['gdp_2021'] = np.where(gdp['gdp_2021'].eq('-'), np.nan, gdp['gdp_2021'])

In [ ]:
# gdp의 열별 결측값 개수를 확인합니다.
# [힌트] isna()와 sum() 함수를 사용하세요.
gdp.isna().sum()

In [ ]:
# gdp의 정보를 확인합니다.
gdp.info()

In [ ]:
# gdp의 처음 5행을 출력합니다.
gdp.head()

In [ ]:
# gdp에서 결측값이 있는 행을 삭제합니다.
gdp = gdp.dropna()

In [ ]:
# gdp의 행 개수를 출력합니다.
# [힌트] shape 속성 결과에 0번 인덱스 원소를 선택하세요.
gdp.shape[0]

In [ ]:
# gdp_2021의 자료형을 정수로 변환합니다.
gdp['gdp_2021'] = gdp['gdp_2021'].astype(int)

In [ ]:
# gdp의 열별 자료형을 확인합니다.
gdp.dtypes

### 데이터 병합

두 데이터프레임을 좌우로 병합할 때 기준이 되는 열을 외래키라고 합니다.<br>
두 외래키에서 일치하는 원소가 없으면 병합했을 때 행 개수가 0일 수 있습니다.<br>
병합 방법은 내부병합, 외부병합, 왼쪽병합 등이 있습니다.<br>
실습 데이터셋 오른쪽에 새로운 열을 추가할 때 왼쪽병합을 주로 사용합니다.

In [ ]:
# 왼쪽 외래키에는 있지만 오른쪽 외래키에 없는 원소를 확인합니다.
# [힌트] set() 함수를 사용하여 시리즈를 집합으로 변환하세요.
# 두 집합으로 차집합을 실행하세요.
set(pvt['국가']) - set(gdp['nation'])

In [ ]:
# pvt와 gdp를 왼쪽병합합니다.
# [힌트] pd.merge() 함수를 사용하세요.
# 왼쪽병합은 how 매개변수에 'left'를 지정합니다.
# 외래키는 on 매개변수에 열이름을 문자열로 지정합니다.
# [참고] 외래키 열이름이 다르면 left_on, right_on 매개변수에 각각 지정해야 합니다.
pd.merge(left = pvt, right = gdp, how = 'left', left_on = '국가', right_on = 'nation')

In [ ]:
# gdp의 열이름 중 'nation'을 '국가', 'gdp_2021'을 '국민소득'으로 변경합니다.
# [힌트] rename() 함수를 사용하세요.
gdp = gdp.rename(columns = {'nation': '국가', 'gdp_2021': '국민소득'})

In [ ]:
# pvt와 gdp를 왼쪽병합하고 mgd에 할당합니다.
mgd = pd.merge(left = pvt, right = gdp, how = 'left', on = '국가')

In [ ]:
# mgd의 정보를 확인합니다.
mgd.info()

In [ ]:
# mgd의 처음 5행을 출력합니다.
mgd.head()

### 연속형 변수의 구간화(binning)

연속형 변수를 기준에 따라 범주형으로 변환하는 것을 구간화(binning)라고 합니다.<br>
구간화는 이상치와 비선형 문제를 해결하고, 결측값을 쉽게 처리할 수 있습니다.<br>
범주가 2개면 np.where(), 3개 이상이면 np.seelct() 함수를 사용하는 것이 좋습니다.

In [ ]:
# 국민소득의 기술통계량을 확인합니다.
# [힌트] describe() 함수를 사용하세요.
mgd['국민소득'].describe()

In [ ]:
# 국민소득이 23,000 달러 이상이면 '선진국', 6,500 ~ 23000 달러 미만이면 '신흥국', 
# 6,500 달러 미만이면 '개도국'인 원소를 갖는 국가등급 열을 생성합니다.
# [힌트] np.select() 함수를 사용하세요.
# condlist 매개변수에 조건, choicelist 매개변수에 원소를 리스트로 지정합니다.
mgd['국가등급'] = np.select(
    condlist = [mgd['국민소득'].ge(23000),
                mgd['국민소득'].ge(10000),
                mgd['국민소득'].lt(10000),
                mgd['국민소득'].isna()],
    choicelist = ['선진국', '신흥국', '개도국', '없음']
)

In [ ]:
# 국가등급의 원소별 빈도수를 확인합니다.
# [힌트] value_counts() 함수를 사용하세요.
mgd['국가등급'].value_counts()

In [ ]:
# 국가등급의 원소별 상대도수를 확인합니다.
# [힌트] value_counts() 함수에 normalize 매개변수를 추가하세요.
mgd['국가등급'].value_counts(normalize = True)

In [ ]:
# 국가등급이 '없음'인 행을 출력합니다.
mgd[mgd['국가등급'].eq('없음')]

### 외부 파일로 저장

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 전역 변수 목록을 출력합니다.
%whos

In [ ]:
# 여러 객체를 하나의 압축 파일로 저장합니다.
# 파일명은 'Seafood_Trade_Prep.z'으로 지정하세요.
# [힌트] joblib.dump() 함수를 사용하세요.
joblib.dump(value = [sfd, mgd], filename = 'Seafood_Trade_Prep.z')

In [ ]:
# 작업 경로에 있는 폴더명과 파일명을 출력합니다.
sorted(os.listdir())

## End of Document